In [1]:
import pandas as pd
import os
from pyspark.sql import SparkSession, functions as F



# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ETL")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/03 23:25:08 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/09/03 23:25:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 23:25:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load in datasets

### Transactions

In [2]:
# 'transactions.parquet' files
path = '../data/tables/transactions_20210228_20210827_snapshot'
transactions_21_02_21_08 = spark.read.parquet(path)

transactions_21_02_21_08.printSchema()


path = '../data/tables/transactions_20210828_20220227_snapshot'
transactions_21_08_22_02 = spark.read.parquet(path)

transactions_21_08_22_02.printSchema()

path = '../data/tables/transactions_20220228_20220828_snapshot'
transactions_22_02_22_08 = spark.read.parquet(path)

transactions_22_02_22_08.printSchema()

transactions_21_02_21_08.limit(10)


root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


### Consumer Data

In [3]:
# 'consumer_fraud_probability.csv' file
path = '../data/tables/consumer_fraud_probability.csv'
cust_fp = spark.read.csv(path, header=True)

cust_fp.printSchema()
cust_fp.limit(10)
# cust_fp.dtypes
# cust_fp.count()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035
16556,2022-02-20,89.65663294494827
10278,2021-09-28,83.59136689427714
15790,2021-12-30,71.77065889280253
5233,2021-08-29,85.87123303878818
230,2021-08-28,86.28328808934151


In [4]:
# 'consumer_user_details.parquet' file
path = '../data/tables/consumer_user_details.parquet'
cust_user_det = spark.read.parquet(path)

cust_user_det.printSchema()
cust_user_det.limit(10)
# cust_user_det.count()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [5]:
# 'tbl_consumer.csv' consumer data file
path = '../data/tables/tbl_consumer.csv'
cust_tbl = spark.read.csv(path, sep='|', header=True)

cust_tbl.printSchema()
cust_tbl.limit(10)
# cust_tbl.dtypes
# cust_tbl.count()

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: string (nullable = true)



name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


### Merchant Data

In [6]:
# 'merchant_fraud_probability.csv' file
path = '../data/tables/merchant_fraud_probability.csv'
merch_fp = spark.read.csv(path, header=True)

merch_fp.printSchema()
merch_fp.limit(10)
# merch_fp.dtypes
# merch_fp.count()

root
 |-- merchant_abn: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



merchant_abn,order_datetime,fraud_probability
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902
31334588839,2021-12-26,38.36165958070444
23686790459,2021-12-10,79.4543441508535
14827550074,2021-11-26,46.45775596795885
31334588839,2021-11-26,36.20971272078342
19492220327,2021-12-18,33.819672154331755


In [7]:
# 'tbl_merchants.csv' merchants data file
path = '../data/tables/tbl_merchants.parquet'
merch_tbl = spark.read.parquet(path)

merch_tbl.printSchema()
merch_tbl.limit(10)

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


### External Data

In [8]:
# 2021 census data CSVs

path = '../data/externaldataset/2021 Census GCP Statistical Area 2 for AUS/*'
sa2_census = spark.read.csv(path, header=True)

sa2_census.limit(10)


23/09/03 23:25:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2_CODE_2021,F_300_399_15_19_yrs,F_300_399_20_24_yrs,F_300_399_25_34_yrs,F_300_399_35_44_yrs,F_300_399_45_54_yrs,F_300_399_55_64_yrs,F_300_399_65_74_yrs,F_300_399_75_84_yrs,F_300_399_85ov,F_300_399_Tot,F_400_499_15_19_yrs,F_400_499_20_24_yrs,F_400_499_25_34_yrs,F_400_499_35_44_yrs,F_400_499_45_54_yrs,F_400_499_55_64_yrs,F_400_499_65_74_yrs,F_400_499_75_84_yrs,F_400_499_85ov,F_400_499_Tot,F_500_649_15_19_yrs,F_500_649_20_24_yrs,F_500_649_25_34_yrs,F_500_649_35_44_yrs,F_500_649_45_54_yrs,F_500_649_55_64_yrs,F_500_649_65_74_yrs,F_500_649_75_84_yrs,F_500_649_85ov,F_500_649_Tot,F_650_799_15_19_yrs,F_650_799_20_24_yrs,F_650_799_25_34_yrs,F_650_799_35_44_yrs,F_650_799_45_54_yrs,F_650_799_55_64_yrs,F_650_799_65_74_yrs,F_650_799_75_84_yrs,F_650_799_85ov,F_650_799_Tot,F_800_999_15_19_yrs,F_800_999_20_24_yrs,F_800_999_25_34_yrs,F_800_999_35_44_yrs,F_800_999_45_54_yrs,F_800_999_55_64_yrs,F_800_999_65_74_yrs,F_800_999_75_84_yrs,F_800_999_85ov,F_800_999_Tot,F_1000_1249_15_19_yrs,F_1000_1249_20_24_yrs,F_1000_1249_25_34_yrs,F_1000_1249_35_44_yrs,F_1000_1249_45_54_yrs,F_1000_1249_55_64_yrs,F_1000_1249_65_74_yrs,F_1000_1249_75_84_yrs,F_1000_1249_85ov,F_1000_1249_Tot,F_1250_1499_15_19_yrs,F_1250_1499_20_24_yrs,F_1250_1499_25_34_yrs,F_1250_1499_35_44_yrs,F_1250_1499_45_54_yrs,F_1250_1499_55_64_yrs,F_1250_1499_65_74_yrs,F_1250_1499_75_84_yrs,F_1250_1499_85ov,F_1250_1499_Tot,F_1500_1749_15_19_yrs,F_1500_1749_20_24_yrs,F_1500_1749_25_34_yrs,F_1500_1749_35_44_yrs,F_1500_1749_45_54_yrs,F_1500_1749_55_64_yrs,F_1500_1749_65_74_yrs,F_1500_1749_75_84_yrs,F_1500_1749_85ov,F_1500_1749_Tot,F_1750_1999_15_19_yrs,F_1750_1999_20_24_yrs,F_1750_1999_25_34_yrs,F_1750_1999_35_44_yrs,F_1750_1999_45_54_yrs,F_1750_1999_55_64_yrs,F_1750_1999_65_74_yrs,F_1750_1999_75_84_yrs,F_1750_1999_85ov,F_1750_1999_Tot,F_2000_2999_15_19_yrs,F_2000_2999_20_24_yrs,F_2000_2999_25_34_yrs,F_2000_2999_35_44_yrs,F_2000_2999_45_54_yrs,F_2000_2999_55_64_yrs,F_2000_2999_65_74_yrs,F_2000_2999_75_84_yrs,F_2000_2999_85ov,F_2000_2999_Tot,F_3000_3499_15_19_yrs,F_3000_3499_20_24_yrs,F_3000_3499_25_34_yrs,F_3000_3499_35_44_yrs,F_3000_3499_45_54_yrs,F_3000_3499_55_64_yrs,F_3000_3499_65_74_yrs,F_3000_3499_75_84_yrs,F_3000_3499_85ov,F_3000_3499_Tot,F_3500_more_15_19_yrs,F_3500_more_20_24_yrs,F_3500_more_25_34_yrs,F_3500_more_35_44_yrs,F_3500_more_45_54_yrs,F_3500_more_55_64_yrs,F_3500_more_65_74_yrs,F_3500_more_75_84_yrs,F_3500_more_85ov,F_3500_more_Tot,F_PI_NS_15_19_yrs,F_PI_NS_ns_20_24_yrs,F_PI_NS_ns_25_34_yrs,F_PI_NS_ns_35_44_yrs,F_PI_NS_ns_45_54_yrs,F_PI_NS_ns_55_64_yrs,F_PI_NS_ns_65_74_yrs,F_PI_NS_ns_75_84_yrs,F_PI_NS_ns_85_yrs_ovr,F_PI_NS_ns_Tot,F_Tot_15_19_yrs,F_Tot_20_24_yrs,F_Tot_25_34_yrs,F_Tot_35_44_yrs,F_Tot_45_54_yrs,F_Tot_55_64_yrs,F_Tot_65_74_yrs,F_Tot_75_84_yrs,F_Tot_85ov,F_Tot_Tot,P_Neg_Nil_income_15_19_yrs,P_Neg_Nil_income_20_24_yrs,P_Neg_Nil_income_25_34_yrs,P_Neg_Nil_income_35_44_yrs,P_Neg_Nil_income_45_54_yrs,P_Neg_Nil_income_55_64_yrs,P_Neg_Nil_income_65_74_yrs,P_Neg_Nil_income_75_84_yrs,P_Negtve_Nil_incme_85_yrs_ovr,P_Neg_Nil_income_Tot,P_1_149_15_19_yrs,P_1_149_20_24_yrs,P_1_149_25_34_yrs,P_1_149_35_44_yrs,P_1_149_45_54_yrs,P_1_149_55_64_yrs,P_1_149_65_74_yrs,P_1_149_75_84_yrs,P_1_149_85ov,P_1_149_Tot,P_150_299_15_19_yrs,P_150_299_20_24_yrs,P_150_299_25_34_yrs,P_150_299_35_44_yrs,P_150_299_45_54_yrs,P_150_299_55_64_yrs,P_150_299_65_74_yrs,P_150_299_75_84_yrs,P_150_299_85ov,P_150_299_Tot,P_300_399_15_19_yrs,P_300_399_20_24_yrs,P_300_399_25_34_yrs,P_300_399_35_44_yrs,P_300_399_45_54_yrs,P_300_399_55_64_yrs,P_300_399_65_74_yrs,P_300_399_75_84_yrs,P_300_399_85ov,P_300_399_Tot,P_400_499_15_19_yrs,P_400_499_20_24_yrs,P_400_499_25_34_yrs,P_400_499_35_44_yrs,P_400_499_45_54_yrs,P_400_499_55_64_yrs,P_400_499_65_74_yrs,P_400_499_75_84_yrs,P_400_499_85ov,P_400_499_Tot,P_500_649_15_19_yrs,P_500_649_20_24_yrs,P_500_649_25_34_yrs,P_500_649_35_44_yrs,P_500_649_45_54_yrs,P_500_649_55_64_yrs,P_500_649_65_74_yrs,P_500_649_75_84_yrs,P_500_649_85ov,P_500_649_Tot
101021007,3,7,3,19,15,38,44,30,3,16

In [9]:
###############################################################################################################################################################################################

In [10]:
# SA2 population data

path = '../data/externaldataset/SA2_Populations_AUS.xlsx'

# Have to manually fix the dataframe as the formatting is messed up

sa2_pops = pd.read_excel(path, sheet_name=1)

# remove empty / useless rows at top
sa2_pops = sa2_pops.iloc[5:,:]

sa2_pops.head()

# drop unnecessary columns
sa2_pops = sa2_pops.drop(sa2_pops.iloc[:, 10:30],axis = 1)

sa2_pops = sa2_pops.drop(sa2_pops.columns[[0,1,2,3,4,6,11]], axis=1)

# fix header
sa2_pops.columns = sa2_pops.iloc[1]

sa2_pops = sa2_pops[3:]

# drop empty rows at bottom of table
sa2_pops = sa2_pops.dropna()

sa2_pops = sa2_pops.rename(columns={'no.': 'population_2021'})

sa2_pops.head()

6,SA4 name,SA3 name,SA2 code,SA2 name,population_2021
8,Capital Region,Queanbeyan,101021007,Braidwood,4332
9,Capital Region,Queanbeyan,101021008,Karabar,8548
10,Capital Region,Queanbeyan,101021009,Queanbeyan,11375
11,Capital Region,Queanbeyan,101021010,Queanbeyan - East,5097
12,Capital Region,Queanbeyan,101021012,Queanbeyan West - Jerrabomberra,12748


In [11]:
###############################################################################################################################################################################################

In [12]:
# Locality to SA2 coding index CSV

path = '../data/externaldataset/2022 Locality to 2021 SA2 Coding Index.csv'

sa2_to_postcode = spark.read.csv(path, header=True)

sa2_to_postcode.limit(10)

LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE_2021,SA2_NAME_2021
ABS195,ACTON PARK,EXTRA LOCALITY,2600,ACT,801051123,Black Mountain
loc6f02f098e82d,ACTON,GAZETTED LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,ANU,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,AUSTRALIAN NATION...,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,SPINNAKER ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc6f02f098e82d,SPRINGBANK ISLAND,ALIAS LOCALITY,2601,ACT,801051049,Acton
loc8af26acb532d,AINSLIE,GAZETTED LOCALITY,2602,ACT,801051050,Ainslie
locf0cacaa1119e,AMAROO,GAZETTED LOCALITY,2914,ACT,801041034,Amaroo
locf94248966b6c,ARANDA,GAZETTED LOCALITY,2614,ACT,801011001,Aranda
ABS358,BELLVIEW,EXTRA LOCALITY,2620,ACT,801071089,Tuggeranong


## 1st Step: Remove useless columns, data type conversions, rename columns where necessary

### Transactions

In [ ]:
#transactions_21_02_21_08 = transactions_21_02_21_08.drop('order_id')

#transactions_21_08_22_02 = transactions_21_08_22_02.drop('order_id')

#transactions_22_02_22_08 = transactions_22_02_22_08.drop('order_id')

transactions_21_02_21_08.printSchema()

transactions_21_08_22_02.printSchema()

transactions_22_02_22_08.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



### Consumer Data

In [13]:
cust_fp = \
cust_fp.withColumn(
    'user_id',
    F.col('user_id').cast('long')
).withColumn(
    'order_datetime',
    F.col('order_datetime').cast('date')
).withColumn(
    'consumer_fraud_probability_%',
    F.col('fraud_probability').cast('double')
).drop('fraud_probability')

cust_fp.printSchema()


root
 |-- user_id: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- consumer_fraud_probability_%: double (nullable = true)



In [14]:
cust_user_det.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



In [ ]:
cust_tbl = \
cust_tbl.withColumn(
    'postcode',
    F.col('postcode').cast('long')
).withColumnRenamed(
    'name',
    'consumer_name'   
).withColumnRenamed(
    'state',
    'consumer_state'   
).withColumnRenamed(
    'postcode',
    'consumer_postcode'   
).withColumnRenamed(
    'gender',
    'consumer_gender'   
).withColumn(
    'consumer_id',
    F.col('consumer_id').cast('long')
).drop('address')

cust_tbl.printSchema()

root
 |-- consumer_name: string (nullable = true)
 |-- consumer_state: string (nullable = true)
 |-- consumer_postcode: long (nullable = true)
 |-- consumer_gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)



### Merchant Data

In [16]:
merch_fp = \
merch_fp.withColumn(
    'merchant_abn',
    F.col('merchant_abn').cast('long')
).withColumn(
    'order_datetime',
    F.col('order_datetime').cast('date')
).withColumn(
    'merchant_fraud_probability_%',
    F.col('fraud_probability').cast('double')
).drop('fraud_probability')

merch_fp.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_fraud_probability_%: double (nullable = true)



In [17]:
merch_tbl = \
merch_tbl.withColumnRenamed(
    'name',
    'merchant_name'
)

merch_tbl.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



### External Data

In [19]:
# 2021 census data CSVs

path = '../data/externaldataset/2021 Census GCP Statistical Area 2 for AUS/*'
sa2_census = spark.read.csv(path, header=True)

sa2_census.limit(10)


SA2_CODE_2021,F_300_399_15_19_yrs,F_300_399_20_24_yrs,F_300_399_25_34_yrs,F_300_399_35_44_yrs,F_300_399_45_54_yrs,F_300_399_55_64_yrs,F_300_399_65_74_yrs,F_300_399_75_84_yrs,F_300_399_85ov,F_300_399_Tot,F_400_499_15_19_yrs,F_400_499_20_24_yrs,F_400_499_25_34_yrs,F_400_499_35_44_yrs,F_400_499_45_54_yrs,F_400_499_55_64_yrs,F_400_499_65_74_yrs,F_400_499_75_84_yrs,F_400_499_85ov,F_400_499_Tot,F_500_649_15_19_yrs,F_500_649_20_24_yrs,F_500_649_25_34_yrs,F_500_649_35_44_yrs,F_500_649_45_54_yrs,F_500_649_55_64_yrs,F_500_649_65_74_yrs,F_500_649_75_84_yrs,F_500_649_85ov,F_500_649_Tot,F_650_799_15_19_yrs,F_650_799_20_24_yrs,F_650_799_25_34_yrs,F_650_799_35_44_yrs,F_650_799_45_54_yrs,F_650_799_55_64_yrs,F_650_799_65_74_yrs,F_650_799_75_84_yrs,F_650_799_85ov,F_650_799_Tot,F_800_999_15_19_yrs,F_800_999_20_24_yrs,F_800_999_25_34_yrs,F_800_999_35_44_yrs,F_800_999_45_54_yrs,F_800_999_55_64_yrs,F_800_999_65_74_yrs,F_800_999_75_84_yrs,F_800_999_85ov,F_800_999_Tot,F_1000_1249_15_19_yrs,F_1000_1249_20_24_yrs,F_1000_1249_25_34_yrs,F_1000_1249_35_44_yrs,F_1000_1249_45_54_yrs,F_1000_1249_55_64_yrs,F_1000_1249_65_74_yrs,F_1000_1249_75_84_yrs,F_1000_1249_85ov,F_1000_1249_Tot,F_1250_1499_15_19_yrs,F_1250_1499_20_24_yrs,F_1250_1499_25_34_yrs,F_1250_1499_35_44_yrs,F_1250_1499_45_54_yrs,F_1250_1499_55_64_yrs,F_1250_1499_65_74_yrs,F_1250_1499_75_84_yrs,F_1250_1499_85ov,F_1250_1499_Tot,F_1500_1749_15_19_yrs,F_1500_1749_20_24_yrs,F_1500_1749_25_34_yrs,F_1500_1749_35_44_yrs,F_1500_1749_45_54_yrs,F_1500_1749_55_64_yrs,F_1500_1749_65_74_yrs,F_1500_1749_75_84_yrs,F_1500_1749_85ov,F_1500_1749_Tot,F_1750_1999_15_19_yrs,F_1750_1999_20_24_yrs,F_1750_1999_25_34_yrs,F_1750_1999_35_44_yrs,F_1750_1999_45_54_yrs,F_1750_1999_55_64_yrs,F_1750_1999_65_74_yrs,F_1750_1999_75_84_yrs,F_1750_1999_85ov,F_1750_1999_Tot,F_2000_2999_15_19_yrs,F_2000_2999_20_24_yrs,F_2000_2999_25_34_yrs,F_2000_2999_35_44_yrs,F_2000_2999_45_54_yrs,F_2000_2999_55_64_yrs,F_2000_2999_65_74_yrs,F_2000_2999_75_84_yrs,F_2000_2999_85ov,F_2000_2999_Tot,F_3000_3499_15_19_yrs,F_3000_3499_20_24_yrs,F_3000_3499_25_34_yrs,F_3000_3499_35_44_yrs,F_3000_3499_45_54_yrs,F_3000_3499_55_64_yrs,F_3000_3499_65_74_yrs,F_3000_3499_75_84_yrs,F_3000_3499_85ov,F_3000_3499_Tot,F_3500_more_15_19_yrs,F_3500_more_20_24_yrs,F_3500_more_25_34_yrs,F_3500_more_35_44_yrs,F_3500_more_45_54_yrs,F_3500_more_55_64_yrs,F_3500_more_65_74_yrs,F_3500_more_75_84_yrs,F_3500_more_85ov,F_3500_more_Tot,F_PI_NS_15_19_yrs,F_PI_NS_ns_20_24_yrs,F_PI_NS_ns_25_34_yrs,F_PI_NS_ns_35_44_yrs,F_PI_NS_ns_45_54_yrs,F_PI_NS_ns_55_64_yrs,F_PI_NS_ns_65_74_yrs,F_PI_NS_ns_75_84_yrs,F_PI_NS_ns_85_yrs_ovr,F_PI_NS_ns_Tot,F_Tot_15_19_yrs,F_Tot_20_24_yrs,F_Tot_25_34_yrs,F_Tot_35_44_yrs,F_Tot_45_54_yrs,F_Tot_55_64_yrs,F_Tot_65_74_yrs,F_Tot_75_84_yrs,F_Tot_85ov,F_Tot_Tot,P_Neg_Nil_income_15_19_yrs,P_Neg_Nil_income_20_24_yrs,P_Neg_Nil_income_25_34_yrs,P_Neg_Nil_income_35_44_yrs,P_Neg_Nil_income_45_54_yrs,P_Neg_Nil_income_55_64_yrs,P_Neg_Nil_income_65_74_yrs,P_Neg_Nil_income_75_84_yrs,P_Negtve_Nil_incme_85_yrs_ovr,P_Neg_Nil_income_Tot,P_1_149_15_19_yrs,P_1_149_20_24_yrs,P_1_149_25_34_yrs,P_1_149_35_44_yrs,P_1_149_45_54_yrs,P_1_149_55_64_yrs,P_1_149_65_74_yrs,P_1_149_75_84_yrs,P_1_149_85ov,P_1_149_Tot,P_150_299_15_19_yrs,P_150_299_20_24_yrs,P_150_299_25_34_yrs,P_150_299_35_44_yrs,P_150_299_45_54_yrs,P_150_299_55_64_yrs,P_150_299_65_74_yrs,P_150_299_75_84_yrs,P_150_299_85ov,P_150_299_Tot,P_300_399_15_19_yrs,P_300_399_20_24_yrs,P_300_399_25_34_yrs,P_300_399_35_44_yrs,P_300_399_45_54_yrs,P_300_399_55_64_yrs,P_300_399_65_74_yrs,P_300_399_75_84_yrs,P_300_399_85ov,P_300_399_Tot,P_400_499_15_19_yrs,P_400_499_20_24_yrs,P_400_499_25_34_yrs,P_400_499_35_44_yrs,P_400_499_45_54_yrs,P_400_499_55_64_yrs,P_400_499_65_74_yrs,P_400_499_75_84_yrs,P_400_499_85ov,P_400_499_Tot,P_500_649_15_19_yrs,P_500_649_20_24_yrs,P_500_649_25_34_yrs,P_500_649_35_44_yrs,P_500_649_45_54_yrs,P_500_649_55_64_yrs,P_500_649_65_74_yrs,P_500_649_75_84_yrs,P_500_649_85ov,P_500_649_Tot
101021007,3,7,3,19,15,38,44,30,3,16

In [20]:
###############################################################################################################################################################################################

In [21]:
# SA2 population data
sa2_pops=spark.createDataFrame(sa2_pops)

In [22]:
sa2_pops.limit(10)

sa2_pops.printSchema()

root
 |-- SA4 name: string (nullable = true)
 |-- SA3 name: string (nullable = true)
 |-- SA2 code: long (nullable = true)
 |-- SA2 name: string (nullable = true)
 |-- population_2021: long (nullable = true)



In [23]:
# convert casings and rename columns where necessary
sa2_pops = \
sa2_pops.withColumnRenamed(
    'SA4 name',
    'sa4_name'
).withColumnRenamed(
    'SA3 name',
    'sa3_name'
).withColumnRenamed(
    'SA2 code',
    'sa2_code'
).withColumnRenamed(
    'SA2 name',
    'sa2_name'
)


In [24]:
###############################################################################################################################################################################################

In [25]:
# Locality to SA2 coding index CSV
sa2_to_postcode.limit(10)

sa2_to_postcode.printSchema()



root
 |-- LOCALITY_ID: string (nullable = true)
 |-- LOCALITY_NAME : string (nullable = true)
 |-- LOCALITY_TYPE: string (nullable = true)
 |-- POSTCODE: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- SA2_MAINCODE_2021: string (nullable = true)
 |-- SA2_NAME_2021: string (nullable = true)



In [26]:
sa2_to_postcode = sa2_to_postcode.select(['POSTCODE', 'SA2_MAINCODE_2021']).withColumn(
    'POSTCODE',
    F.col('POSTCODE').cast('long')
).withColumn(
    'SA2_MAINCODE_2021',
    F.col('SA2_MAINCODE_2021').cast('long')
).withColumnRenamed(
    'SA2_MAINCODE_2021',
    'sa2_code'
)

sa2_to_postcode = sa2_to_postcode.toDF(*[c.lower() for c in sa2_to_postcode.columns])

## 2nd Step: Cleaning, feature engineering, aggregation

### Transactions

In [ ]:
transactions_all = transactions_21_02_21_08.union(transactions_21_08_22_02).union(transactions_22_02_22_08)

In [ ]:
transactions_all.sort('order_datetime').head(3)

NameError: name 'transactions_all' is not defined

In [ ]:
transactions_all.sort('order_datetime').tail(3)

[Row(user_id=11136, merchant_abn=63685007785, dollar_value=48.12988669206644, order_id='a6194c61-ae7a-4037-a425-2dd7cda42fb7', order_datetime=datetime.date(2022, 10, 26)),
 Row(user_id=11137, merchant_abn=85502310765, dollar_value=4.768751754248673, order_id='72a3639e-25d5-4bcc-9e8b-26e737b8c0e4', order_datetime=datetime.date(2022, 10, 26)),
 Row(user_id=11138, merchant_abn=43186523025, dollar_value=35.12766841919401, order_id='3f3b1d40-e8ec-4d4b-ac08-5f273586523c', order_datetime=datetime.date(2022, 10, 26))]

In [ ]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = transactions_all

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()
df = df.dropDuplicates(['order_id'])

# 3. Validate user_id and merchant_abn
df = df.filter((F.col('user_id') > 0) & (F.col('merchant_abn') > 0))

# 4. Validate dollar value
df = df.filter(F.col('dollar_value') > 0)

# 5. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))

transactions_all_clean = df.drop('order_id')

In [ ]:
transactions_all_clean

user_id,merchant_abn,dollar_value,order_datetime
83,46804135891,40.88736687931136,2021-08-20
18574,40252040480,110.20656881653261,2021-08-20
18653,17324645993,6.067842686819487,2021-08-20
215,81219314324,66.32075609316571,2021-08-20
18860,77013874702,2.082645905098512,2021-08-20
18881,29616684420,91.12197214702582,2021-08-20
399,71674475255,34.45156578850835,2021-08-20
646,24852446429,44.287278647072334,2021-08-20
19179,95492562735,43.393091988896295,2021-08-20
19230,11437621158,259.7314287658183,2021-08-20


### Consumer Data

In [27]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_user_det

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate consumer_id and user_id
df = df.filter((F.col('consumer_id') > 0) & (F.col('user_id') > 0))

cust_user_det_clean = df

In [28]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_tbl

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate consumer_id
df = df.filter((F.col('consumer_id') > 0))

# 4. Validate consumer_name 
df = df.filter(F.col('consumer_name').rlike("[a-zA-Z][a-zA-Z ]+"))

# 5. Validate consumer_state 
df = df.filter(F.col('consumer_state').isin(['NT','ACT','SA','TAS','WA','QLD','VIC','NSW']))

# 6. Validate consumer_postcode
df = df.filter(F.col('consumer_postcode').between(200, 9729))

# 7. Validate consumer_gender
df = df.filter(F.col('consumer_gender').isin(['Undisclosed', 'Male', 'Female']))

cust_tbl_clean = df
               
            

In [29]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

df = cust_fp

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate user_id
df = df.filter((F.col('user_id') > 0))

# 4. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))

# 5. Validate consumer fraud probability
df = df.filter(F.col('consumer_fraud_probability_%').between(0, 100))

cust_fp_clean = df

In [30]:
cust_fp_clean.sort('user_id')

user_id,order_datetime,consumer_fraud_probability_%
1,2022-02-20,9.805431136520959
2,2021-09-25,10.069850934775245
2,2021-08-30,9.599513915425788
3,2021-11-03,8.300636455314633
4,2021-10-09,9.633302411090419
5,2022-01-11,27.496186536467164
5,2022-02-08,9.02022421158597
5,2021-10-04,10.868364868449886
6,2021-12-12,10.459280127078758
9,2021-12-13,10.58055311139687


### Merchant Data

In [31]:
merch_tbl.count()

4026

In [32]:
# Extract different features from tags (words, revenue band, take rate)
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType, ArrayType

# Define a UDF to extract the sections within square and normal brackets
def split_tags(input_tags):
    sep_tags = re.findall(r'[\[\(\{]([^\[\]\(\)\{\}]*)[\]\)\}]', input_tags)
    sep_tags_without_brackets = [re.sub(r'[\[\]\(\)\{\}]', '', tag) for tag in sep_tags]
        
    return sep_tags_without_brackets

# Define a UDF to extract "take rate" numbers
def extract_take_rate(input_rate):
    match = re.search(r'take rate: ([\d.]+)', input_rate)
    if match:
        return float(match.group(1))
    return None

# Register the UDFs
split_tags_udf = udf(split_tags, ArrayType(StringType()))

clean_take_rate_udf = udf(extract_take_rate, FloatType())

# Apply the UDF to the DataFrame
merch_tbl_clean = merch_tbl.withColumn("sep_tags", split_tags_udf(merch_tbl["tags"]))

# Create separate columns each segment of the tags
merch_tbl_clean = merch_tbl_clean.withColumn("words", merch_tbl_clean["sep_tags"].getItem(0))
merch_tbl_clean = merch_tbl_clean.withColumn("revenue_level", merch_tbl_clean["sep_tags"].getItem(1))
merch_tbl_clean = merch_tbl_clean.withColumn("take_rate_%", merch_tbl_clean["sep_tags"].getItem(2))

merch_tbl_clean = merch_tbl_clean.withColumn("take_rate_%", clean_take_rate_udf(merch_tbl_clean["take_rate_%"]))



In [33]:
merch_tbl_clean = merch_tbl_clean.drop('tags', 'sep_tags')

merch_tbl_clean.limit(10)

merchant_name,merchant_abn,words,revenue_level,take_rate_%
Felis Limited,10023283211,"furniture, home f...",e,0.18
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",b,4.22
Nunc Sed Company,10165489824,"jewelry, watch, c...",b,4.4
Ultricies Digniss...,10187291046,"wAtch, clock, and...",b,3.29
Enim Condimentum PC,10192359162,music shops - mus...,a,6.33
Fusce Company,10206519221,"gift, card, novel...",a,6.34
Aliquam Enim Inco...,10255988167,"computers, comPUt...",b,4.32
Ipsum Primis Ltd,10264435225,"watch, clock, and...",c,2.39
Pede Ultrices Ind...,10279061213,computer programm...,a,5.71
Nunc Inc.,10323485998,"furniture, home f...",a,6.61


In [34]:
merch_tbl_clean.count()

4026

In [35]:
df = merch_tbl_clean 

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate merchant_abn
df = df.filter((F.col('merchant_abn') > 0))

# 4. Validate merchant_name 
df = df.filter(F.col('merchant_name').rlike("[a-zA-Z][a-zA-Z ]+"))

# 5. Validate words (SPELL CHECK?) # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)
                                   # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 
#df = df.filter(F.col('words') in ['NT','ACT','SA','TAS','WA','QLD','VIC','NSW'])

# 6. Validate take_rate
df = df.filter((F.col('take_rate_%').between(0, 100)))

# 7. Validate revenue_level
df = df.filter(F.col('revenue_level').isin(['a', 'b', 'c', 'd', 'e']))


merch_tbl_clean = df


In [36]:
df = merch_fp

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate merchant_abn
df = df.filter((F.col('merchant_abn') > 0))

# 4. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
df = df.filter(F.col('order_datetime').between(start_date, end_date))

# 5. Validate merchant fraud probability
df = df.filter(F.col('merchant_fraud_probability_%').between(0, 100))

merch_fp_clean = df


### External Data

In [42]:
# 2021 census data CSVs

path = '../data/externaldataset/2021 Census GCP Statistical Area 2 for AUS/*'
sa2_census = spark.read.csv(path, header=True)

sa2_census.limit(10)


SA2_CODE_2021,F_300_399_15_19_yrs,F_300_399_20_24_yrs,F_300_399_25_34_yrs,F_300_399_35_44_yrs,F_300_399_45_54_yrs,F_300_399_55_64_yrs,F_300_399_65_74_yrs,F_300_399_75_84_yrs,F_300_399_85ov,F_300_399_Tot,F_400_499_15_19_yrs,F_400_499_20_24_yrs,F_400_499_25_34_yrs,F_400_499_35_44_yrs,F_400_499_45_54_yrs,F_400_499_55_64_yrs,F_400_499_65_74_yrs,F_400_499_75_84_yrs,F_400_499_85ov,F_400_499_Tot,F_500_649_15_19_yrs,F_500_649_20_24_yrs,F_500_649_25_34_yrs,F_500_649_35_44_yrs,F_500_649_45_54_yrs,F_500_649_55_64_yrs,F_500_649_65_74_yrs,F_500_649_75_84_yrs,F_500_649_85ov,F_500_649_Tot,F_650_799_15_19_yrs,F_650_799_20_24_yrs,F_650_799_25_34_yrs,F_650_799_35_44_yrs,F_650_799_45_54_yrs,F_650_799_55_64_yrs,F_650_799_65_74_yrs,F_650_799_75_84_yrs,F_650_799_85ov,F_650_799_Tot,F_800_999_15_19_yrs,F_800_999_20_24_yrs,F_800_999_25_34_yrs,F_800_999_35_44_yrs,F_800_999_45_54_yrs,F_800_999_55_64_yrs,F_800_999_65_74_yrs,F_800_999_75_84_yrs,F_800_999_85ov,F_800_999_Tot,F_1000_1249_15_19_yrs,F_1000_1249_20_24_yrs,F_1000_1249_25_34_yrs,F_1000_1249_35_44_yrs,F_1000_1249_45_54_yrs,F_1000_1249_55_64_yrs,F_1000_1249_65_74_yrs,F_1000_1249_75_84_yrs,F_1000_1249_85ov,F_1000_1249_Tot,F_1250_1499_15_19_yrs,F_1250_1499_20_24_yrs,F_1250_1499_25_34_yrs,F_1250_1499_35_44_yrs,F_1250_1499_45_54_yrs,F_1250_1499_55_64_yrs,F_1250_1499_65_74_yrs,F_1250_1499_75_84_yrs,F_1250_1499_85ov,F_1250_1499_Tot,F_1500_1749_15_19_yrs,F_1500_1749_20_24_yrs,F_1500_1749_25_34_yrs,F_1500_1749_35_44_yrs,F_1500_1749_45_54_yrs,F_1500_1749_55_64_yrs,F_1500_1749_65_74_yrs,F_1500_1749_75_84_yrs,F_1500_1749_85ov,F_1500_1749_Tot,F_1750_1999_15_19_yrs,F_1750_1999_20_24_yrs,F_1750_1999_25_34_yrs,F_1750_1999_35_44_yrs,F_1750_1999_45_54_yrs,F_1750_1999_55_64_yrs,F_1750_1999_65_74_yrs,F_1750_1999_75_84_yrs,F_1750_1999_85ov,F_1750_1999_Tot,F_2000_2999_15_19_yrs,F_2000_2999_20_24_yrs,F_2000_2999_25_34_yrs,F_2000_2999_35_44_yrs,F_2000_2999_45_54_yrs,F_2000_2999_55_64_yrs,F_2000_2999_65_74_yrs,F_2000_2999_75_84_yrs,F_2000_2999_85ov,F_2000_2999_Tot,F_3000_3499_15_19_yrs,F_3000_3499_20_24_yrs,F_3000_3499_25_34_yrs,F_3000_3499_35_44_yrs,F_3000_3499_45_54_yrs,F_3000_3499_55_64_yrs,F_3000_3499_65_74_yrs,F_3000_3499_75_84_yrs,F_3000_3499_85ov,F_3000_3499_Tot,F_3500_more_15_19_yrs,F_3500_more_20_24_yrs,F_3500_more_25_34_yrs,F_3500_more_35_44_yrs,F_3500_more_45_54_yrs,F_3500_more_55_64_yrs,F_3500_more_65_74_yrs,F_3500_more_75_84_yrs,F_3500_more_85ov,F_3500_more_Tot,F_PI_NS_15_19_yrs,F_PI_NS_ns_20_24_yrs,F_PI_NS_ns_25_34_yrs,F_PI_NS_ns_35_44_yrs,F_PI_NS_ns_45_54_yrs,F_PI_NS_ns_55_64_yrs,F_PI_NS_ns_65_74_yrs,F_PI_NS_ns_75_84_yrs,F_PI_NS_ns_85_yrs_ovr,F_PI_NS_ns_Tot,F_Tot_15_19_yrs,F_Tot_20_24_yrs,F_Tot_25_34_yrs,F_Tot_35_44_yrs,F_Tot_45_54_yrs,F_Tot_55_64_yrs,F_Tot_65_74_yrs,F_Tot_75_84_yrs,F_Tot_85ov,F_Tot_Tot,P_Neg_Nil_income_15_19_yrs,P_Neg_Nil_income_20_24_yrs,P_Neg_Nil_income_25_34_yrs,P_Neg_Nil_income_35_44_yrs,P_Neg_Nil_income_45_54_yrs,P_Neg_Nil_income_55_64_yrs,P_Neg_Nil_income_65_74_yrs,P_Neg_Nil_income_75_84_yrs,P_Negtve_Nil_incme_85_yrs_ovr,P_Neg_Nil_income_Tot,P_1_149_15_19_yrs,P_1_149_20_24_yrs,P_1_149_25_34_yrs,P_1_149_35_44_yrs,P_1_149_45_54_yrs,P_1_149_55_64_yrs,P_1_149_65_74_yrs,P_1_149_75_84_yrs,P_1_149_85ov,P_1_149_Tot,P_150_299_15_19_yrs,P_150_299_20_24_yrs,P_150_299_25_34_yrs,P_150_299_35_44_yrs,P_150_299_45_54_yrs,P_150_299_55_64_yrs,P_150_299_65_74_yrs,P_150_299_75_84_yrs,P_150_299_85ov,P_150_299_Tot,P_300_399_15_19_yrs,P_300_399_20_24_yrs,P_300_399_25_34_yrs,P_300_399_35_44_yrs,P_300_399_45_54_yrs,P_300_399_55_64_yrs,P_300_399_65_74_yrs,P_300_399_75_84_yrs,P_300_399_85ov,P_300_399_Tot,P_400_499_15_19_yrs,P_400_499_20_24_yrs,P_400_499_25_34_yrs,P_400_499_35_44_yrs,P_400_499_45_54_yrs,P_400_499_55_64_yrs,P_400_499_65_74_yrs,P_400_499_75_84_yrs,P_400_499_85ov,P_400_499_Tot,P_500_649_15_19_yrs,P_500_649_20_24_yrs,P_500_649_25_34_yrs,P_500_649_35_44_yrs,P_500_649_45_54_yrs,P_500_649_55_64_yrs,P_500_649_65_74_yrs,P_500_649_75_84_yrs,P_500_649_85ov,P_500_649_Tot
101021007,3,7,3,19,15,38,44,30,3,16

In [43]:
###############################################################################################################################################################################################

In [44]:
sa2_pops.limit(10)

sa2_pops.printSchema()

root
 |-- sa4_name: string (nullable = true)
 |-- sa3_name: string (nullable = true)
 |-- sa2_code: long (nullable = true)
 |-- sa2_name: string (nullable = true)
 |-- population_2021: long (nullable = true)



In [45]:
df = sa2_pops

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# 3. Validate population
df = df.filter(F.col('population_2021') >= 0)

sa2_pops_clean = df


In [46]:
###############################################################################################################################################################################################

In [47]:
df = sa2_to_postcode

# 1. Check for Null Values
df = df.na.drop()

# 2. Check for Duplicates
df = df.dropDuplicates()

# No need to validate postcode as inner join will be performed with validated postcodes

sa2_to_postcode_clean=df

sa2_to_postcode_clean.sort('postcode').limit(10)



postcode,sa2_code
800,701011002
810,701021029
810,701021026
810,701021027
810,701021016
810,701021023
810,701021018
810,701021010
810,701021024
810,701021013


### Aggregation

In [48]:
output_relative_dir = '../data/curated/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
   

# now, for each type of data set we will need, we will create the paths
for target_dir in ('consumer', 'merchant'): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)


In [49]:
#cons_combined = 

In [50]:
#cons_combined.limit(5)

In [51]:
#cons_combined.write.parquet("../data/curated/consumer/consumer_combined.parquet")

In [52]:
#merch_combined

In [53]:
#merch_combined.limit(5)

In [54]:
#merch_combined.write.parquet("../data/curated/merchant/merchant_combined.parquet")

In [55]:
orig_combined = cust_tbl_clean.join(cust_user_det_clean, on='consumer_id', how='inner') \
.join(transactions_all_clean, on='user_id', how ='inner') \
.join(cust_fp_clean, on=['user_id', 'order_datetime'], how='left').na.fill(0) \
.join(merch_tbl_clean, on='merchant_abn', how='inner') \
.join(merch_fp_clean, on=['merchant_abn', 'order_datetime'], how='left').na.fill(0)



In [56]:
sa2_combined = sa2_to_postcode_clean.join(sa2_pops_clean, on='sa2_code', how='inner').withColumnRenamed('population_2021','sa2_population_2021')


In [57]:
all_combined = orig_combined.join(sa2_combined.withColumnRenamed('postcode','consumer_postcode'), on='consumer_postcode', how='inner')

In [58]:
all_combined = all_combined.withColumnRenamed(
    'sa2_code',
    'consumer_sa2_code'
).withColumnRenamed(
    'sa4_name',
    'consumer_sa4_name'
).withColumnRenamed(
    'sa3_name',
    'consumer_sa3_name'
).withColumnRenamed(
    'sa2_name',
    'consumer_sa2_name'
).withColumnRenamed(
    'sa2_population_2021',
    'consumer_sa2_pop_2021'
).withColumnRenamed(
    'words',
    'merchant_description'
).withColumnRenamed(
    'revenue_level',
    'merchant_revenue_level'
).withColumnRenamed(
    'take_rate_%',
    'merchant_take_rate_%'
)



In [59]:
all_combined.limit(5)

consumer_postcode,merchant_abn,order_datetime,user_id,consumer_id,consumer_name,consumer_state,consumer_gender,dollar_value,consumer_fraud_probability_%,merchant_name,merchant_description,merchant_revenue_level,merchant_take_rate_%,merchant_fraud_probability_%,consumer_sa2_code,consumer_sa4_name,consumer_sa3_name,consumer_sa2_name,consumer_sa2_pop_2021
4606,49505931725,2022-10-13,7,511685,Andrea Jones,QLD,Female,80.56308158918402,0.0,Suspendisse Ac As...,digital goods: bo...,b,4.7,0.0,319021506,Wide Bay,Burnett,Kingaroy Surround...,9615
4606,80518954462,2021-04-11,7,511685,Andrea Jones,QLD,Female,248.259198693175,0.0,Neque Sed Dictum ...,"computers, compUt...",b,3.49,0.0,319021506,Wide Bay,Burnett,Kingaroy Surround...,9615
4606,20885454195,2021-06-29,7,511685,Andrea Jones,QLD,Female,106.09604494465586,0.0,Pharetra Ut Indus...,"cable, satellite,...",b,4.94,0.0,319021506,Wide Bay,Burnett,Kingaroy Surround...,9615
4606,78327477486,2022-05-24,7,511685,Andrea Jones,QLD,Female,250.1700381157457,0.0,Auctor Inc.,lawn and garden s...,c,1.47,0.0,319021506,Wide Bay,Burnett,Kingaroy Surround...,9615
4606,90173050473,2022-03-14,7,511685,Andrea Jones,QLD,Female,108.27880211271156,0.0,Lobortis Class In...,lawn and garden s...,c,2.49,0.0,319021506,Wide Bay,Burnett,Kingaroy Surround...,9615


In [61]:
all_combined.write.mode('overwrite').parquet("../data/curated/all_data_combined.parquet")